## Here I want to do several things
The first routine should take a given a set of points in 3D and compute the list of all possible tetrahedrons.

The second routine should take a second set of 3D points and the list of all tetrahedrons computed before and computes their barycentric coordinates for the tetrahedrons containing them.

I did refresh my memory by reading this http://www.iue.tuwien.ac.at/phd/nentchev/node31.html

In [1]:
from scipy.spatial import ConvexHull
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
import scipy.spatial

### List of all possible tethrahedrons
The first routine should take a given a set of points in 3D and compute the list of all possible tetrahedrons.

In [2]:
#generate some random test points
N_A = 10 # like 10 points...
set_3D_points_A = np.random.rand(N_A,3) # all values between [0,1]

print set_3D_points_A[0:3,:], np.shape(set_3D_points_A) # only show the first 3 points

[[ 0.33018085  0.05523432  0.27419375]
 [ 0.66869361  0.04049841  0.2866533 ]
 [ 0.92849717  0.04446711  0.92596894]] (10, 3)


We start with a definition, a tetrahedron is made of four points/vertex in 3D spaces, it has four triangle faces.

So if I have *N* points, then all possible combinations of *4* of them give me the list of possible tetrahedrons.

The function below tries to do so using all possible combinations.

In [3]:
def fun_list_all_tetrahedron_from_set_of_points(set_3D_points):
    """
    The function does what its name says.
    It takes a list of a least four points in 3D space and return
    all possible combinations of four points defining a tetrahedron.
    """
    # get the list of indexs for each tetrahedron
    N = np.shape(set_3D_points)[0]
    vec_N = np.arange(0,N)
    
    # get the coordinates for each list of tetrahedra 
    nb_combibation = itertools.combinations(vec_N, 4)
    list_combination = np.array(list(nb_combibation))
    #tetrahedron_coordinates = set_3D_points[nb_combibation,:]
    return list_combination


**nb_combibation** gives the index numbers for each group of four points defining a tetrahedron.

This approach didn't test if for a set of four points the points were in the same plan and didn't tell if the tetrahedrons were exclusive or not in the sens no points were located within a tetrahedron. You may want to know how many unique tetrahedron there are in your set of points.

Some test data below to try the function.

In [4]:
list_index_tetra = fun_list_all_tetrahedron_from_set_of_points(set_3D_points_A)
print len(list_index_tetra),list_index_tetra[0:5,:], np.shape(list_index_tetra)
print "For a set of %1.0f points we have %1.0f tetrahedronds" % (N_A,np.shape(list_index_tetra)[0])

210 [[0 1 2 3]
 [0 1 2 4]
 [0 1 2 5]
 [0 1 2 6]
 [0 1 2 7]] (210, 4)
For a set of 10 points we have 210 tetrahedronds


## Compute the barycentric coordinates for the tetrahedrons
The second routine should take a second set of 3D points and the list of all tetrahedrons computed before and computes their barycentric coordinates for the tetrahedrons containg them.

In [5]:
N_B = 20
set_3D_points_B = np.random.rand(N_B,3) # all values between [0,1]
set_3D_points_B = np.vstack([set_3D_points_B, [0.5, 0.5, 0.5]])
print set_3D_points_B[-3:,:]

[[ 0.52810739  0.16081564  0.29618339]
 [ 0.285631    0.96736037  0.77310344]
 [ 0.5         0.5         0.5       ]]


I basically use the scipy spatial function here... What I do is the Delaunay triangulation to *clean* the number of good tetrahedrons. It also makes it easier to use the function from scipy.

In [6]:
def fun_compute_barycentric_coordinates(tetrahedron_points, targets_points):
    """
    The function take one tetrahedron as input and a list of points.
    For each point we want to know if it is located in the this tetrahedron, 
    and if yes we want the barycentric coordinates.
    """
    res_bcoords = np.zeros((np.shape(targets_points)[0],4))
    #print np.shape(res_bcoords)
    
    # Delaunay triangulation
    tri = scipy.spatial.Delaunay(tetrahedron_points)
    print tri.simplices
    #print np.shape(targets_points)
    for ii in np.arange(np.shape(targets_points)[0]):
        point = targets_points[ii,:]
        tetrahedra = tri.find_simplex(point)
        if tetrahedra >=  0: 
            # point located in the tetrahedron
            try:
                res_bc = fun_get_barycentric_coordinates(tetrahedra, tri, point)
                print ii, res_bc
                res_bcoords[ii,:] = res_bc
            except ValueError:
                #print ii#, point, np.array([-1, -1, -1])
                res_bcoords[ii,:] = np.array([-1, -1, -1, -1])
        else:
            res_bcoords[ii,:] = np.array([-1, -1, -1, -1])# * np.ones((4,1))
            #print ii,# point, np.array([-1, -1, -1])
    return res_bcoords
    
def fun_get_barycentric_coordinates(tetrahedra, trig, point):
    """
    Here we solve the equation to get the barycentric coordinates where
    tetrahedra are the points defining the testrahedron
    point are the point coordinates in cartersian space
    trig the result of the Delaunay triangulation
    """
    # find the 
    X = trig.transform[tetrahedra,:3]
    Y = point - trig.transform[tetrahedra,3]
    b = np.einsum('ijk,ik->ij', X, Y)
    b_coords = np.c_[b, 1 - b.sum(axis=1)]
    print b_coords
    return b_coords

In [7]:
for ii in np.arange(len(list_index_tetra)):
    tetra_points = set_3D_points_A[list_index_tetra[ii,:],:]
    #print tetra_points
    bary_coords = fun_compute_barycentric_coordinates(tetra_points, set_3D_points_B)
    #print bary_coords

[[1 3 2 0]]
[[1 3 2 0]]
[[1 3 2 0]]
[[1 3 2 0]]
[[1 3 2 0]]
[[1 3 2 0]]
[[1 3 2 0]]
[[2 3 1 0]]
[[3 2 1 0]]
[[2 3 1 0]]
[[2 1 3 0]]
[[2 1 3 0]]
[[2 3 1 0]]
[[3 2 1 0]]
[[2 3 1 0]]
[[2 1 3 0]]
[[2 1 3 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[2 1 3 0]]
[[2 1 3 0]]
[[2 3 1 0]]
[[2 1 3 0]]
[[2 1 3 0]]
[[2 3 1 0]]
[[2 1 3 0]]
[[1 3 2 0]]
[[1 3 2 0]]
[[2 3 1 0]]
[[3 2 1 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[3 2 1 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[3 2 1 0]]
[[3 2 1 0]]
[[2 3 1 0]]
[[3 2 1 0]]
[[2 3 1 0]]
[[2 3 1 0]]
[[1 2 3 0]]
[[2 1 3 0]]
[[1 2 3 0]]
[[1 2 3 0]]
[[2 3 1 0]]
[[1 3 2 0]]
[[1 2 3 0]]
[[1 2 3 0]]
[[1 3 2 0]]
[[1 2 3 0]]
[[1 2 3 0]]
[[3 1 2 0]]
[[1 2 3 0]]
[[1 3 2 0]]
[[1 3 2 0]]
[[1 3 2 0]]
[[2 1 3 0]]
[[2 1 3 0]]
[[1 3 2 0]]
[[1 2 3 0]]
[[1 2 3 0]]
[[1 3 2 0]]
[[2 1 3 0]]
[[1 3 2 0]]
[[1 3 2 0]]
[[2 1 3 0]]
[[1 2 3 0]]
[[2 3 1 0]]
[[2 1 3 0]]
[[1 3 2 0]]
[[1 3 2 0]]
[[2 1 3 0]]
[[1 3 2 0]]
[[1 3 2 0]]
[[1 

In [8]:
# Delaunay triangulation
tri = scipy.spatial.Delaunay(set_3D_points_A)
print "I have %1.0f tetrahedra after the triangulation. " % np.shape(tri.simplices)[0]

I have 18 tetrahedra after the triangulation. 


In [9]:
# find which point are in which tetrahedron
tetrahedra = tri.find_simplex(set_3D_points_B)
print tetrahedra

# find the 
X = tri.transform[tetrahedra,:3]
Y = set_3D_points_B - tri.transform[tetrahedra,3]
b = np.einsum('ijk,ik->ij', X, Y)
bcoords = np.c_[b, 1 - b.sum(axis=1)]
print bcoords


[-1 -1 -1  8 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  6 -1 -1]
[[-6.24154489  6.38297508  0.96964555 -0.11107574]
 [ 2.1328507   1.31615557 -1.84562237 -0.60338389]
 [-4.17840569  5.43218686  0.79560383 -1.04938499]
 [ 0.1430933   0.49812239  0.05622559  0.30255872]
 [ 2.69347403 -0.90526679 -0.00659835 -0.78160889]
 [ 0.38656141  3.50288318 -0.35081545 -2.53862913]
 [ 1.93139622  0.06841523  1.28573441 -2.28554586]
 [-0.86952881  2.30872699 -0.66034565  0.22114747]
 [ 1.6497277   0.16626364  0.69351441 -1.50950575]
 [-2.04391896  3.18516594  0.92839065 -1.06963763]
 [ 3.99325543 -0.61534011  0.23296127 -2.61087659]
 [ 0.86293822 -1.12177266  2.58162648 -1.32279204]
 [ 5.27405122 -3.15302678  0.43320398 -1.55422841]
 [ 0.02149493  1.29802458  0.03906901 -0.35858852]
 [ 1.81806441 -0.34840582 -0.66098261  0.19132402]
 [ 4.11031079 -3.28369334  0.82624391 -0.65286136]
 [-1.29716523  4.02796689 -0.70275337 -1.02804829]
 [-0.86910802  3.78821223  0.24069635 -2.15980057]
 [ 0.2598286   0.

In [10]:
# display the barycentric coordinate for each point in its corresponding tetrahedra
for tt in np.arange(len(tetrahedra)):
    if tetrahedra[tt] >= 0: # then the point is in a tetrahedron
        print "indexes of tetradron"
        print tri.simplices[tetrahedra[tt],:], set_3D_points_B[tt,:]
        print set_3D_points_A[tri.simplices[tetrahedra[tt],:],:]

indexes of tetradron
[8 3 1 2] [ 0.71669857  0.09239212  0.75771266]
[[ 0.76580803  0.27747846  0.72889897]
 [ 0.57936357  0.0741903   0.71696209]
 [ 0.66869361  0.04049841  0.2866533 ]
 [ 0.92849717  0.04446711  0.92596894]]
indexes of tetradron
[5 6 4 0] [ 0.52810739  0.16081564  0.29618339]
[[ 0.61496151  0.06239464  0.20953875]
 [ 0.59276964  0.33779181  0.52725709]
 [ 0.50572595  0.09850981  0.04710101]
 [ 0.33018085  0.05523432  0.27419375]]
